# remote require test

Check that invocations like

```javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
  }
});
```

work as expected.

In [ ]:
# based on https://codepen.io/SitePoint/pen/wWJXWa

import jp_proxy_widget

class D3BarWidget(jp_proxy_widget.JSProxyWidget):
    
    is_ready = False
    
    def __init__(self, *pargs, **kwargs):
        super(D3BarWidget, self).__init__(*pargs, **kwargs)
        self.uses_require(self.make_barchart)
        
    def make_barchart(self):
        # Initialize the widget using they asynchronously loaded "saveAs" module
        self.js_init("""
        element.empty();
        
        // make a div and put dummy text in it for now
        var target = $("<div/>").appendTo(element);
        target.css("background-color", "#ff0");
        
        // configure d3
        element.requirejs.config({
          paths: {
              d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
          }
        });
        
        // Use require to asynchronously load the "saveAs" module.
        element.requirejs(["d3"], function(d3) {
            var interpolator = d3.interpolateRgb(
                                  d3.rgb('#ff99aa'),
                                  d3.rgb('#558855')
                                );
            var get_color = function(d) {
                return interpolator(d/50.0)
            }
            d3.select(target[0])
              .selectAll("div")
              .data([4, 8, 15, 16, 23, 42])
              .enter()
              .append("div")
              .style("display", "inline-block")
              //.style("background", "#4285f4")
              .style("background", (d)=> get_color(d))
              .style("width", "20px")
              .style("margin-right", "3px")
              .style("height", (d)=> d + "px");
            ready();
        });
        """, ready=self.ready)
        
    def ready(self):
        self.is_ready = True
        

d3bar = D3BarWidget()
d3bar

In [ ]:
from jp_proxy_widget import notebook_test_helpers

validators = notebook_test_helpers.ValidationSuite()

In [ ]:
def validate_d3bar():
    assert d3bar.is_ready
    print ("d3bar seems good!")
    
validators.add_validation(d3bar, validate_d3bar)

validators.run_all_in_widget()